In [1]:
import $ivy.`org.apache.spark::spark-sql:3.2.0` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.10.1`

import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

val spark = SparkSession
      .builder()
      .master("local[*]")
      .getOrCreate()

import spark.implicits._

Logger.getRootLogger().setLevel(Level.ERROR)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/05/12 18:58:02 INFO SparkContext: Running Spark version 3.2.0
22/05/12 18:58:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/12 18:58:02 INFO ResourceUtils: ==============================================================
22/05/12 18:58:02 INFO ResourceUtils: No custom resources configured for spark.driver.
22/05/12 18:58:02 INFO ResourceUtils: ==============================================================
22/05/12 18:58:02 INFO SparkContext: Submitted application: 0eac3d19-f007-47d4-a18f-55ebc593121d
22/05/12 18:58:02 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
22/05/1

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                               


import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@5a69721d
import spark.implicits._



In [2]:
val files = List("circuits", "constructor_results", "constructors", "constructor_standings",
                 "drivers", "driver_standings", "drivers_constr_season", "lap_times", "pit_stops",
                 "qualifying", "races", "results", "seasons", "status")

files: List[String] = List(
  "circuits",
  "constructor_results",
  "constructors",
  "constructor_standings",
  "drivers",
  "driver_standings",
  "drivers_constr_season",
  "lap_times",
  "pit_stops",
  "qualifying",
  "races",
  "results",
  "seasons",
  "status"
)

In [10]:
val partCol = Map(
    "circuits" -> "noPartitionCol",
    "constructor_results" -> "noPartitionCol",
    "constructors" -> "noPartitionCol",
    "constructor_standings" -> "noPartitionCol",
    "drivers" -> "noPartitionCol",
    "driver_standings" -> "noPartitionCol",
    "drivers_constr_season" -> "noPartitionCol",
    "lap_times" -> "driverId",
    "pit_stops" -> "noPartitionCol",
    "qualifying" -> "noPartitionCol",
    "races" -> "noPartitionCol",
    "results" -> "noPartitionCol",
    "seasons" -> "noPartitionCol",
    "status" -> "noPartitionCol"
)

partCol: Map[String, String] = Map(
  "pit_stops" -> "noPartitionCol",
  "drivers" -> "noPartitionCol",
  "races" -> "noPartitionCol",
  "constructor_results" -> "noPartitionCol",
  "constructors" -> "noPartitionCol",
  "driver_standings" -> "noPartitionCol",
  "results" -> "noPartitionCol",
  "status" -> "noPartitionCol",
  "drivers_constr_season" -> "noPartitionCol",
  "lap_times" -> "driverId",
  "circuits" -> "noPartitionCol",
  "constructor_standings" -> "noPartitionCol",
  "qualifying" -> "noPartitionCol",
  "seasons" -> "noPartitionCol"
)

In [11]:
def toParquet(tableName: String, partitionCol: String): Unit = {
    val ini = spark.read.format("csv")
        .option("header", "true")
        .option("sep", ",")
        .load(s"../data/$tableName.csv")
    
    if (partitionCol == "noPartitionCol") {
        ini.write.mode("overwrite")
            .parquet(s"../data/parquet/$tableName.parquet")
    } else {
        ini.repartition(col(partitionCol))
            .write.mode("overwrite")
            .partitionBy(partitionCol)
            .parquet(s"../data/parquet/$tableName.parquet")
    }
}

defined function toParquet

In [8]:
def toParquetNoPart(tableName: String): Unit = {
    spark.read.format("csv")
        .option("header", "true")
        .option("sep", ",")
        .load(s"../data/$tableName.csv")
        .write.mode("overwrite")
        .parquet(s"../data/parquetnopart/$tableName.parquet")
}

defined function toParquetNoPart

In [12]:
files.foreach(x => toParquet(x, partCol(x)))

In [9]:
files.foreach(x => toParquetNoPart(x))

In [4]:
spark.read.format("csv")
        .option("header", "true")
        .option("sep", ",")
        .load(s"../data/police/NYPD_Complaint_Data_Historic.csv").printSchema

root
 |-- CMPLNT_NUM: string (nullable = true)
 |-- CMPLNT_FR_DT: string (nullable = true)
 |-- CMPLNT_FR_TM: string (nullable = true)
 |-- CMPLNT_TO_DT: string (nullable = true)
 |-- CMPLNT_TO_TM: string (nullable = true)
 |-- RPT_DT: string (nullable = true)
 |-- KY_CD: string (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- PD_CD: string (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- CRM_ATPT_CPTD_CD: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- JURIS_DESC: string (nullable = true)
 |-- BORO_NM: string (nullable = true)
 |-- ADDR_PCT_CD: string (nullable = true)
 |-- LOC_OF_OCCUR_DESC: string (nullable = true)
 |-- PREM_TYP_DESC: string (nullable = true)
 |-- PARKS_NM: string (nullable = true)
 |-- HADEVELOPT: string (nullable = true)
 |-- X_COORD_CD: string (nullable = true)
 |-- Y_COORD_CD: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Lat_Lon: string (nullable

In [14]:
spark.read.format("csv")
        .option("header", "true")
        .option("sep", ",")
        .load(s"../data/drivers.csv")
        .select("driverId").distinct().count()

res13: Long = 854L